In [1]:
import pandas as pd
import numpy as np
import spacy
# Load the model
nlp = spacy.load("en_core_web_md")

In [3]:
research_areas = pd.read_csv("geography_research_areas.csv", engine = "python")
research_areas.head()

,Relevant Research Area
0,how land-use changes affect local climates wit...
1,examination of historical linkages between Sou...
2,"urban, social-cultural and tourism geographies"
3,general theme of geographic information scienc...
4,Transnational migration and citizenship\n'Dias...


In [5]:
# preprocess
# reformat list into proper sentences that are separated by '.'
research_areas["Relevant Research Area"] = research_areas["Relevant Research Area"].apply(
    lambda x: '. '.join(y.lstrip('()0123456789.-') for y in x.split('\n')))
# convert everything to lowercase
research_areas["Relevant Research Area"] = research_areas["Relevant Research Area"].str.lower()
research_areas.head()

,Relevant Research Area
0,how land-use changes affect local climates wit...
1,examination of historical linkages between sou...
2,"urban, social-cultural and tourism geographies"
3,general theme of geographic information scienc...
4,transnational migration and citizenship. 'dias...


In [7]:
# parse research areas using Spacy to tokens
research_areas["nlp"] = research_areas["Relevant Research Area"].apply(nlp)
research_areas.head()

,Relevant Research Area,nlp
0,how land-use changes affect local climates wit...,"(how, land, -, use, changes, affect, local, cl..."
1,examination of historical linkages between sou...,"(examination, of, historical, linkages, betwee..."
2,"urban, social-cultural and tourism geographies","(urban, ,, social, -, cultural, and, tourism, ..."
3,general theme of geographic information scienc...,"(general, theme, of, geographic, information, ..."
4,transnational migration and citizenship. 'dias...,"(transnational, migration, and, citizenship, ...."


In [8]:
# get noun phrases to compare similarities of noun phrases
def get_keywords(research):
    ls = []
    research_words = list(map(lambda x:x.pos_, research))
    i = len(research)-1

    while i >= 0:
        if research_words[i] in ["PROPN", "NOUN"]:
            j = i-1
            while j >=0 and research_words[j] in ["PROPN", "ADJ"]:
                j -= 1
            ls.append(research[j+1:i+1])
            i = j
        i = i-1
    return ls

In [9]:
research_areas["keywords"] = research_areas["nlp"].apply(get_keywords)
research_areas.head()

,Relevant Research Area,nlp,keywords
0,how land-use changes affect local climates wit...,"(how, land, -, use, changes, affect, local, cl...","[(sensitive, urban, design), (such, knowledge)..."
1,examination of historical linkages between sou...,"(examination, of, historical, linkages, betwee...","[(twentieth, century), (middle, decades), (sea..."
2,"urban, social-cultural and tourism geographies","(urban, ,, social, -, cultural, and, tourism, ...",[(geographies)]
3,general theme of geographic information scienc...,"(general, theme, of, geographic, information, ...","[(spatiotemporal, applications), (infrastructu..."
4,transnational migration and citizenship. 'dias...,"(transnational, migration, and, citizenship, ....","[(cosmopolitanism), (politics), (emotional, ge..."


In [10]:
def get_similarity(keywords_1, keywords_2):
    score = []
    if len(keywords_2) > 0:
        for n in keywords_1:
            s = []
            for i in keywords_2:
                if len(i) > 0:
                    s.append(n.similarity(i))
            score.append(max(s))
    score = list(filter(lambda x:x != 0, score))
    if len(score) > 0:
        return sum(score)/len(score)
    return 0

In [11]:
# compare the similarities of each row to the first row
research_areas["keyword_score"] = research_areas["keywords"].apply(lambda x: get_similarity(research_areas["keywords"][0], x))
research_areas.head()

,Relevant Research Area,nlp,keywords,keyword_score
0,how land-use changes affect local climates wit...,"(how, land, -, use, changes, affect, local, cl...","[(sensitive, urban, design), (such, knowledge)...",1.000000
1,examination of historical linkages between sou...,"(examination, of, historical, linkages, betwee...","[(twentieth, century), (middle, decades), (sea...",0.488276
2,"urban, social-cultural and tourism geographies","(urban, ,, social, -, cultural, and, tourism, ...",[(geographies)],0.218397
3,general theme of geographic information scienc...,"(general, theme, of, geographic, information, ...","[(spatiotemporal, applications), (infrastructu...",0.549076
4,transnational migration and citizenship. 'dias...,"(transnational, migration, and, citizenship, ....","[(cosmopolitanism), (politics), (emotional, ge...",0.426206


In [13]:
research_areas.sort_values(by = "keyword_score", ascending = False).head(10)

,Relevant Research Area,nlp,keywords,keyword_score
0,how land-use changes affect local climates wit...,"(how, land, -, use, changes, affect, local, cl...","[(sensitive, urban, design), (such, knowledge)...",1.000000
41,the impact of climate change and rapid develo...,"( , the, impact, of, climate, change, and, rap...","[(security), (safe, water), (access), (groundw...",0.702280
14,"biogeography, which studies where organisms di...","(biogeography, ,, which, studies, where, organ...","[(their, probable, consequences), (changes), (...",0.696103
150,changing political economy of environment and ...,"(changing, political, economy, of, environment...","[(conservation), (political, context), (change...",0.656424
35,causes of past and future global climate chang...,"(causes, of, past, and, future, global, climat...","[(policies), (national, climate), (ecological,...",0.653951
22,"how radiation interacts with vegetation, how w...","(how, radiation, interacts, with, vegetation, ...","[(degradation), (deforestation), (agriculture)...",0.630954
10,"an international, interdisciplinary study of r...","(an, international, ,, interdisciplinary, stud...","[(routledge), (success), (shadows), (asian, de...",0.628886
7,the applications of gis/rs in watershed and la...,"(the, applications, of, gis, /, rs, in, waters...","[(large, asian, rivers), (flux), (sediment), (...",0.619959
26,earth surface processes and landform morphodyn...,"(earth, surface, processes, and, landform, mor...","[(explanation), (prediction), (models), (role)...",0.619881
85,cities as sites of innovation and creativity r...,"(cities, as, sites, of, innovation, and, creat...","[(innovation), (national, systems), (urban, ec...",0.612662


In [15]:
for sent in bme.nlp[4].sents:
    print(sent)

transnational migration and citizenship.
'diaspora' strategies and extraterritorial citizenship.
asian forced migration.
emotional geographies.
politics of cosmopolitanism


In [14]:
# calculate similarities using word2vec now
from gensim.models.phrases import Phrases, Phraser
from spacy.en.language_data import STOP_WORDS

In [57]:
# function to remove stop words and separate each sentence into a list
def separate_sentence(sentences):
    result = list()
    for sentence in sentences.sents:
        result.append(list(token.lemma_ for token in sentence if token.text not in STOP_WORDS and token.is_punct == False))
    return result

In [58]:
research_areas["sentences"] = research_areas["nlp"].apply(separate_sentence)
research_areas.head()

,Relevant Research Area,nlp,keywords,keyword_score,sentences,sentences_bigram,word2vec_score,final_score
0,how land-use changes affect local climates wit...,"(how, land, -, use, changes, affect, local, cl...","[(sensitive, urban, design), (such, knowledge)...",1.000000,"[[land, use, change, affect, local, climate, p...","[[land, use, changes, affect, local, climates,...",1.000000,1.000000
1,examination of historical linkages between sou...,"(examination, of, historical, linkages, betwee...","[(twentieth, century), (middle, decades), (sea...",0.488276,"[[examination, historical, linkage, southeast,...","[[examination, historical, linkages, southeast...",0.504676,0.496476
2,"urban, social-cultural and tourism geographies","(urban, ,, social, -, cultural, and, tourism, ...",[(geographies)],0.218397,"[[urban, social, cultural, tourism, geography]]","[[urban, social, cultural, tourism, geographies]]",0.210421,0.214409
3,general theme of geographic information scienc...,"(general, theme, of, geographic, information, ...","[(spatiotemporal, applications), (infrastructu...",0.549076,"[[general, theme, geographic, information, sci...","[[general, theme, geographic, information, sci...",0.694887,0.621981
4,transnational migration and citizenship. 'dias...,"(transnational, migration, and, citizenship, ....","[(cosmopolitanism), (politics), (emotional, ge...",0.426206,"[[transnational, migration, citizenship], [dia...","[[transnational, migration, citizenship], [dia...",0.340159,0.383183


In [59]:
# use gensim phrases and phrases to model bigram
phrases = Phrases(research_areas["sentences"].sum())
bigram = Phraser(phrases)
research_areas["sentences_bigram"] = research_areas["sentences"].apply(lambda doc: [bigram[sentence] for sentence in doc])
research_areas.head()

,Relevant Research Area,nlp,keywords,keyword_score,sentences,sentences_bigram,word2vec_score,final_score
0,how land-use changes affect local climates wit...,"(how, land, -, use, changes, affect, local, cl...","[(sensitive, urban, design), (such, knowledge)...",1.000000,"[[land, use, change, affect, local, climate, p...","[[land, use, change, affect, local, climate, p...",1.000000,1.000000
1,examination of historical linkages between sou...,"(examination, of, historical, linkages, betwee...","[(twentieth, century), (middle, decades), (sea...",0.488276,"[[examination, historical, linkage, southeast,...","[[examination, historical, linkage, southeast,...",0.504676,0.496476
2,"urban, social-cultural and tourism geographies","(urban, ,, social, -, cultural, and, tourism, ...",[(geographies)],0.218397,"[[urban, social, cultural, tourism, geography]]","[[urban, social, cultural, tourism, geography]]",0.210421,0.214409
3,general theme of geographic information scienc...,"(general, theme, of, geographic, information, ...","[(spatiotemporal, applications), (infrastructu...",0.549076,"[[general, theme, geographic, information, sci...","[[general, theme, geographic, information, sci...",0.694887,0.621981
4,transnational migration and citizenship. 'dias...,"(transnational, migration, and, citizenship, ....","[(cosmopolitanism), (politics), (emotional, ge...",0.426206,"[[transnational, migration, citizenship], [dia...","[[transnational, migration, citizenship], [dia...",0.340159,0.383183


In [68]:
# train word2vec using the bigram sentences
from gensim.models import Word2Vec
model = Word2Vec(research_areas["sentences_bigram"].sum(), min_count = 1, iter = 200)
print(model)

Word2Vec(vocab=967, size=100, alpha=0.025)


In [69]:
# test the model
model.wv.most_similar('climate_change', topn = 10)

[('rapid', 0.8480092287063599),
 ('uptake', 0.8467185497283936),
 ('delivery;.', 0.8126136064529419),
 ('tracer', 0.809782087802887),
 ('passive', 0.7897651791572571),
 ('balkan', 0.7846596240997314),
 ('degradation', 0.7601733207702637),
 ('scenario', 0.7559983730316162),
 ('soil', 0.7321529388427734),
 ('deforestation', 0.7268885970115662)]

In [70]:
model.wv.most_similar("water")

[('hydro', 0.9268509149551392),
 ('atmosphere', 0.8969287872314453),
 ('geomorphology', 0.8952786326408386),
 ('air', 0.8785158395767212),
 ('access', 0.865626335144043),
 ('safe', 0.8595972657203674),
 ('meet', 0.8581376075744629),
 ('interface', 0.8569030165672302),
 ('geomorphological', 0.856734573841095),
 ('morphodynamic', 0.8493137359619141)]

In [71]:
# calculate the cosine similarities between the different sentences in each row
# if the research area text contains more than 1 sentence, compare the sentence by pair for all combinations
# and take the max score
def get_n_similarity(sentences_1, sentences_2):
    score = 0
    for s1 in sentences_1:
        for s2 in sentences_2:
            temp_score = model.wv.n_similarity(s1, s2)
            if temp_score > score:
                score = temp_score
    return score

In [72]:
research_areas["word2vec_score"] = research_areas["sentences_bigram"].apply(
    lambda x: get_n_similarity(research_areas["sentences_bigram"][0], x))
research_areas.head()

,Relevant Research Area,nlp,keywords,keyword_score,sentences,sentences_bigram,word2vec_score,final_score
0,how land-use changes affect local climates wit...,"(how, land, -, use, changes, affect, local, cl...","[(sensitive, urban, design), (such, knowledge)...",1.000000,"[[land, use, change, affect, local, climate, p...","[[land, use, change, affect, local, climate, p...",1.000000,1.000000
1,examination of historical linkages between sou...,"(examination, of, historical, linkages, betwee...","[(twentieth, century), (middle, decades), (sea...",0.488276,"[[examination, historical, linkage, southeast,...","[[examination, historical, linkage, southeast,...",0.565980,0.496476
2,"urban, social-cultural and tourism geographies","(urban, ,, social, -, cultural, and, tourism, ...",[(geographies)],0.218397,"[[urban, social, cultural, tourism, geography]]","[[urban, social, cultural, tourism, geography]]",0.256250,0.214409
3,general theme of geographic information scienc...,"(general, theme, of, geographic, information, ...","[(spatiotemporal, applications), (infrastructu...",0.549076,"[[general, theme, geographic, information, sci...","[[general, theme, geographic, information, sci...",0.674322,0.621981
4,transnational migration and citizenship. 'dias...,"(transnational, migration, and, citizenship, ....","[(cosmopolitanism), (politics), (emotional, ge...",0.426206,"[[transnational, migration, citizenship], [dia...","[[transnational, migration, citizenship], [dia...",0.285421,0.383183


In [73]:
research_areas.sort_values(by = "word2vec_score", ascending = False).head(10)

,Relevant Research Area,nlp,keywords,keyword_score,sentences,sentences_bigram,word2vec_score,final_score
0,how land-use changes affect local climates wit...,"(how, land, -, use, changes, affect, local, cl...","[(sensitive, urban, design), (such, knowledge)...",1.000000,"[[land, use, change, affect, local, climate, p...","[[land, use, change, affect, local, climate, p...",1.000000,1.000000
88,mixing in ocean general circulation models eff...,"(mixing, in, ocean, general, circulation, mode...","[(scenarios), (climate), (uptake), (experiment...",0.495724,"[[mix, ocean, general, circulation, model, eff...","[[mix, ocean, general, circulation, model, eff...",0.818654,0.664551
22,"how radiation interacts with vegetation, how w...","(how, radiation, interacts, with, vegetation, ...","[(degradation), (deforestation), (agriculture)...",0.630954,"[[radiation, interact, vegetation, model, unde...","[[radiation, interact, vegetation, model, unde...",0.811016,0.718830
142,"driving mechanisms, spatial complexity and en...","( , driving, mechanisms, ,, spatial, complexit...","[(variability), (environmental, impacts), (spa...",0.467450,"[[ , drive, mechanism, spatial, complexity, e...","[[ , drive, mechanism, spatial, complexity, e...",0.805730,0.582802
44,"urban land and housing. gender, families and h...","(urban, land, and, housing, ., gender, ,, fami...","[(chile), (mexico), (america), (industrial, he...",0.529148,"[[urban, land, housing], [gender, family, hous...","[[urban, land, housing], [gender, family, hous...",0.756653,0.665021
123,study landform-sediment assemblages (glacial ...,"( , study, landform, -, sediment, assemblages,...","[(past), (environmental, parameters), (masses)...",0.540900,"[[ , study, landform, sediment, assemblage, gl...","[[ , study, landform, sediment, assemblage, gl...",0.747298,0.686201
66,"glacial history of the atlas mountains, north ...","(glacial, history, of, the, atlas, mountains, ...","[(areas), (wales), (periglacial, landforms), (...",0.600624,"[[glacial, history, atlas, mountain, north, af...","[[glacial, history, atlas, mountain, north, af...",0.741753,0.682409
35,causes of past and future global climate chang...,"(causes, of, past, and, future, global, climat...","[(policies), (national, climate), (ecological,...",0.653951,"[[cause, past, future, global, climate, change...","[[cause, past, future, global, climate_change,...",0.737637,0.710972
47,understand how the physical processes regulati...,"(understand, how, the, physical, processes, re...","[(measurements), (models), (instruments), (var...",0.596980,"[[understand, physical, process, regulate, wat...","[[understand, physical, process, regulate, wat...",0.737151,0.678475
65,the role of geomorphological processes in the ...,"(the, role, of, geomorphological, processes, i...","[(cycle), (geomorphological, processes), (role)]",0.439367,"[[role, geomorphological, process, terrestrial...","[[role, geomorphological, process, terrestrial...",0.731730,0.592873


In [74]:
# take final score as average of 2 scores
research_areas["final_score"] = (research_areas["keyword_score"] + research_areas["word2vec_score"])/2
research_areas.head()

,Relevant Research Area,nlp,keywords,keyword_score,sentences,sentences_bigram,word2vec_score,final_score
0,how land-use changes affect local climates wit...,"(how, land, -, use, changes, affect, local, cl...","[(sensitive, urban, design), (such, knowledge)...",1.000000,"[[land, use, change, affect, local, climate, p...","[[land, use, change, affect, local, climate, p...",1.000000,1.000000
1,examination of historical linkages between sou...,"(examination, of, historical, linkages, betwee...","[(twentieth, century), (middle, decades), (sea...",0.488276,"[[examination, historical, linkage, southeast,...","[[examination, historical, linkage, southeast,...",0.565980,0.527128
2,"urban, social-cultural and tourism geographies","(urban, ,, social, -, cultural, and, tourism, ...",[(geographies)],0.218397,"[[urban, social, cultural, tourism, geography]]","[[urban, social, cultural, tourism, geography]]",0.256250,0.237323
3,general theme of geographic information scienc...,"(general, theme, of, geographic, information, ...","[(spatiotemporal, applications), (infrastructu...",0.549076,"[[general, theme, geographic, information, sci...","[[general, theme, geographic, information, sci...",0.674322,0.611699
4,transnational migration and citizenship. 'dias...,"(transnational, migration, and, citizenship, ....","[(cosmopolitanism), (politics), (emotional, ge...",0.426206,"[[transnational, migration, citizenship], [dia...","[[transnational, migration, citizenship], [dia...",0.285421,0.355813


In [75]:
research_areas.sort_values(by = "final_score", ascending = False)["Relevant Research Area"].head()

0     how land-use changes affect local climates wit...
22    how radiation interacts with vegetation, how w...
35    causes of past and future global climate chang...
14    biogeography, which studies where organisms di...
41     the impact of climate change and rapid develo...
Name: Relevant Research Area, dtype: object

In [96]:
from gensim.corpora import Dictionary
from gensim.models import ldamodel
from gensim import similarities

In [151]:
dictionary = Dictionary([sum(sentences, []) for sentences in research_areas["sentences_bigram"]])
corpus = [dictionary.doc2bow(sum(sentences, [])) for sentences in research_areas["sentences_bigram"]]

In [152]:
np.random.seed(124) # setting random seed to get the same results each time.
model_lda = ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=25)
model_lda.show_topics()

[(21,
  '0.016*"dynamic" + 0.014*"national" + 0.013*"migration" + 0.012*"politic" + 0.012*" " + 0.012*"development" + 0.012*"modern" + 0.012*"south" + 0.012*"regional" + 0.012*"north"'),
 (3,
  '0.030*"global" + 0.030*"carbon" + 0.030*"climate_change" + 0.019*"remote_sense" + 0.015*"policy" + 0.015*"land" + 0.015*"human" + 0.015*"model" + 0.015*"national" + 0.015*"monitor"'),
 (13,
  '0.018*" " + 0.018*"crustacean" + 0.018*"anti" + 0.014*"earth" + 0.013*"change" + 0.011*"development" + 0.011*"particular" + 0.010*"use" + 0.009*"study" + 0.009*"biogeography"'),
 (20,
  '0.049*"urban" + 0.020*"geography" + 0.018*"system" + 0.014*"especially" + 0.014*"financial" + 0.014*"gpns" + 0.014*"asia" + 0.014*"life" + 0.013*"environment" + 0.012*"city"'),
 (15,
  '0.024*"change" + 0.018*"development" + 0.016*"interest" + 0.012*"research" + 0.012*"geography" + 0.012*"study" + 0.012*"conservation" + 0.012*"political" + 0.012*"range" + 0.012*"water"'),
 (9,
  '0.039*"urban" + 0.028*"geography" + 0.020*

In [153]:
sum(research_areas["sentences_bigram"][1], [])

['examination',
 'historical',
 'linkage',
 'southeast',
 'asia',
 'port',
 'city',
 'liverpool',
 'life',
 'malay',
 'man',
 'arrive',
 'seafarer',
 'middle',
 'decade',
 'twentieth',
 'century']

In [163]:
doc = dictionary.doc2bow(sum(research_areas["sentences_bigram"][0], []))
doc_lda = model_lda[doc]

In [165]:
# find the most similar documents from all the documents in the corpus
index = similarities.MatrixSimilarity(model_lda[corpus])

# print similarities (cosine similarity)
research_areas["lda_score"] = index[doc_lda]
research_areas.head()

,Relevant Research Area,nlp,keywords,keyword_score,sentences,sentences_bigram,word2vec_score,final_score,lda_score
0,how land-use changes affect local climates wit...,"(how, land, -, use, changes, affect, local, cl...","[(sensitive, urban, design), (such, knowledge)...",1.000000,"[[land, use, change, affect, local, climate, p...","[[land, use, change, affect, local, climate, p...",1.000000,1.000000,1.0
1,examination of historical linkages between sou...,"(examination, of, historical, linkages, betwee...","[(twentieth, century), (middle, decades), (sea...",0.488276,"[[examination, historical, linkage, southeast,...","[[examination, historical, linkage, southeast,...",0.565980,0.527128,0.0
2,"urban, social-cultural and tourism geographies","(urban, ,, social, -, cultural, and, tourism, ...",[(geographies)],0.218397,"[[urban, social, cultural, tourism, geography]]","[[urban, social, cultural, tourism, geography]]",0.256250,0.237323,0.0
3,general theme of geographic information scienc...,"(general, theme, of, geographic, information, ...","[(spatiotemporal, applications), (infrastructu...",0.549076,"[[general, theme, geographic, information, sci...","[[general, theme, geographic, information, sci...",0.674322,0.611699,1.0
4,transnational migration and citizenship. 'dias...,"(transnational, migration, and, citizenship, ....","[(cosmopolitanism), (politics), (emotional, ge...",0.426206,"[[transnational, migration, citizenship], [dia...","[[transnational, migration, citizenship], [dia...",0.285421,0.355813,0.0


In [166]:
research_areas.sort_values(by = "lda_score", ascending = False)

,Relevant Research Area,nlp,keywords,keyword_score,sentences,sentences_bigram,word2vec_score,final_score,lda_score
0,how land-use changes affect local climates wit...,"(how, land, -, use, changes, affect, local, cl...","[(sensitive, urban, design), (such, knowledge)...",1.000000,"[[land, use, change, affect, local, climate, p...","[[land, use, change, affect, local, climate, p...",1.000000,1.000000,1.000000
145,"challenge of inequality, addressing themes as ...","(challenge, of, inequality, ,, addressing, the...","[(crime), (fear), (health), (housing), (reside...",0.425836,"[[challenge, inequality, address, theme, diver...","[[challenge, inequality, address, theme, diver...",0.496367,0.461102,1.000000
19,importance of materials and technologies in po...,"(importance, of, materials, and, technologies,...","[(field), (materials), (lively, behaviour), (p...",0.562830,"[[importance, material, technology, political,...","[[importance, material, technology, political,...",0.534820,0.548825,1.000000
3,general theme of geographic information scienc...,"(general, theme, of, geographic, information, ...","[(spatiotemporal, applications), (infrastructu...",0.549076,"[[general, theme, geographic, information, sci...","[[general, theme, geographic, information, sci...",0.674322,0.611699,1.000000
109,migration indonesia feminist theory,"(migration, indonesia, feminist, theory)","[(feminist, theory), (migration)]",0.369802,"[[migration, indonesia, feminist, theory]]","[[migration, indonesia, feminist, theory]]",0.267479,0.318640,1.000000
106,politics of planning and development feminist ...,"(politics, of, planning, and, development, fem...","[(articulations), (culture), (feminist), (plan...",0.431978,"[[politic, planning, development, feminist, cr...","[[politic, planning, development, feminist, cr...",0.296860,0.364419,0.923441
23,the development of algorithms for locational a...,"(the, development, of, algorithms, for, locati...","[(sdsss), (hci), (interaction), (new, algorith...",0.525430,"[[development, algorithms, locational, analysi...","[[development, algorithms, locational, analysi...",0.686283,0.605857,0.477174
37,human geography,"(human, geography)","[(human, geography)]",0.449437,"[[human, geography]]","[[human, geography]]",0.427611,0.438524,0.019518
59,biogeography; conservation,"(biogeography, ;, conservation)","[(conservation), (biogeography)]",0.347280,"[[biogeography, conservation]]","[[biogeography, conservation]]",0.479412,0.413346,0.019518
63,energy and cities,"(energy, and, cities)","[(cities), (energy)]",0.468221,"[[energy, city]]","[[energy, city]]",0.636690,0.552456,0.019518
